In [ ]:
import ast
from hashlib import md5
from Crypto.Cipher import AES
import os

# ================= 1. 分析 Vecs 的线性结构 =================
print("[*] Analyzing the linear structure of vecs...")

# 将 vecs (字符串列表) 转换为 GF(2) 矩阵
# vecs 元素如 '0111...'
vecs_matrix = Matrix(GF(2), [[int(c) for c in s] for s in vecs])

# 取第一个向量作为基准偏移量 (Base Vector)
base_vec = vecs_matrix[0]

# 计算差分矩阵: {v - v0}
diff_vecs = [row - base_vec for row in vecs_matrix]
diff_matrix = Matrix(GF(2), diff_vecs)

# 计算线性基 (Basis)
basis = diff_matrix.row_space()
basis_vectors = basis.basis()

DIM = len(basis_vectors)
BLOCK_LEN = vecs_matrix.ncols() # 应该是 16
NUM_BLOCKS = 50

print(f"[+] Block Length: {BLOCK_LEN}")
print(f"[+] Subspace Dimension: {DIM}") # 预期是 12

# ================= 2. 构建线性方程组 =================
print("[*] Building linear system...")

# 读取 output.txt
with open("output.txt", "r") as f:
    content = f.read()
    if "🎁 :" in content:
        list_str = content.split("🎁 :")[1].split("\n")[0].strip()
    else:
        list_str = content.strip()
    data = ast.literal_eval(list_str)

M = []
B = []

for nonce_int, target_bit in data:
    row = []
    
    # rhs_val 用于存储方程右边的值 (target - 常数项)
    rhs_val = int(target_bit)
    
    # 遍历 50 个 block
    for block_idx in range(NUM_BLOCKS):
        # 提取当前 block 对应的 Nonce 片段 (16 bits)
        # 移位逻辑：Block 0 是最高位
        shift_amount = (NUM_BLOCKS - 1 - block_idx) * BLOCK_LEN
        nonce_chunk = (nonce_int >> shift_amount) & ((1 << BLOCK_LEN) - 1)
        
        # 将 nonce_chunk 转为 Sage 向量
        # 注意位序：字符串 index 0 是最高位
        nonce_vec = vector(GF(2), [(nonce_chunk >> (BLOCK_LEN - 1 - i)) & 1 for i in range(BLOCK_LEN)])
        
        # 1. 计算常数部分的贡献 (Offset Contribution)
        # dot_product 结果是 GF(2) 元素，必须转为 int
        const_contribution = int(nonce_vec * base_vec)
        
        # 更新 RHS: 减去常数项 (在 GF(2) 中减法等于加法)
        rhs_val = (rhs_val + const_contribution) % 2
        
        # 2. 计算变量系数 (Coefficients of Basis)
        for basis_vec in basis_vectors:
            # 计算 nonce 对该基向量的投影
            coeff = int(nonce_vec * basis_vec)
            row.append(coeff)
            
    M.append(row)
    B.append(rhs_val)

# ================= 3. 求解 =================
print("[*] Solving with SageMath...")
MS = Matrix(GF(2), M)
VS = vector(GF(2), B)

try:
    # solve_right 求解 M * x = B
    coeffs = MS.solve_right(VS)
    print("[+] Solution found!")
    
    # ================= 4. 重组 Key =================
    print("[*] Reconstructing Key...")
    
    full_key_bits = ""
    
    # 遍历 50 个 Block，利用解出的系数重建每个 Block
    for block_idx in range(NUM_BLOCKS):
        # 取出当前 Block 对应的 DIM (12) 个系数
        block_coeffs = coeffs[block_idx*DIM : (block_idx+1)*DIM]
        
        # 重建向量: V = Base + Sum(c_i * B_i)
        recovered_vec = base_vec
        for i in range(DIM):
            if block_coeffs[i] == 1:
                recovered_vec += basis_vectors[i]
        
        # 将向量转回 '01' 字符串
        vec_str = "".join(str(x) for x in recovered_vec)
        full_key_bits += vec_str
        
    key_int = int(full_key_bits, 2)
    print(f"[+] Recovered Key: {key_int}")
    
    # ================= 5. 解密 Flag =================
    aes_key = md5(str(key_int).encode()).digest()

    ciphertext = b'r\x9f\xa5N\x19\xa3\x8b-\xfen\xa7\xc9\xca\x87\x04\x873#ju\xc5\x06\xd0\x85L\x1ab\xa1XEre\xe5\x8c:h'
    cipher = AES.new(key=aes_key, nonce=b"Tiffany", mode=AES.MODE_CTR)
    flag = cipher.decrypt(ciphertext)
    print(f"🚩 FLAG: {flag}")

except ValueError:
    print("[-] No solution found. System might be inconsistent.")
except Exception as e:
    print(f"[-] An error occurred during reconstruction/decryption: {e}")

[*] Analyzing the linear structure of vecs...
[+] Block Length: 16
[+] Subspace Dimension: 12
[*] Building linear system...
[*] Solving with SageMath...
[+] Solution found!
[*] Reconstructing Key...
[+] Recovered Key: 5269527612119407587786353615157795985417908133651612604766820343808126017578868036657851247004349463574077479045988376689591315526021713985224829634362726087928609624897665402116323513972430505674943664565324446295412414008890459633008091924
🚩 FLAG: b'LILCTF{Un1qUe_s0luti0n_1N_sUbSp4C3!}'
